In [28]:
!which python

/opt/conda/envs/myenv310/bin/python


In [29]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [30]:
!python -V

Python 3.10.18


In [31]:
import pickle
import pandas as pd

In [32]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [33]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [34]:
year = 2023
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [35]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [36]:
import numpy as np
np.std(y_pred)

np.float64(6.247488852238703)

In [37]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [38]:
df_result = df[['ride_id']].copy()
df_result['prediction'] = y_pred
output_file = f'yellow_tripdata_{year:04d}-{month:02d}_predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [43]:
!ls -lh

total 66M
-rw-rw-rw- 1 codespace codespace  17K Jun 16 07:11 model.bin
-rw-rw-rw- 1 codespace codespace 5.1K Jun 16 10:26 starter.ipynb
-rw-rw-rw- 1 codespace codespace 1.7K Jun 16 09:56 starter.py
-rw-rw-rw- 1 codespace codespace  66M Jun 16 10:48 yellow_tripdata_2023-03_predictions.parquet
